In [1]:
import pandas as pd
import re
import numpy as np
import pybliometrics
from pybliometrics.scopus import ScopusSearch
from pybliometrics.scopus import AbstractRetrieval
pybliometrics.scopus.init()

In [8]:
ab = AbstractRetrieval("2-s2.0-84929582121", view = 'FULL')
s = ScopusSearch("2-s2.0-84929582121")


In [9]:


ab.references

[Reference(position='1', id='66249088017', doi=None, title='The second wave of synthetic biology: From modules to systems', authors='Purnick, P.E.M.; Weiss, R.', authors_auid=None, authors_affiliationid=None, sourcetitle='Nat. Rev. Mol. Cell Biol.', publicationyear='2009', coverDate=None, volume=None, issue=None, first=None, last=None, citedbycount=None, type=None, text=None, fulltext='Purnick, P. E. M. and Weiss, R. (2009) The second wave of synthetic biology: From modules to systems Nat. Rev. Mol. Cell Biol. 10, 410 - 422'),
 Reference(position='2', id='33746436302', doi=None, title=None, authors='Alon, U.', authors_auid=None, authors_affiliationid=None, sourcetitle='An Introduction to Systems Biology: Design Principles of Biological Circuits', publicationyear='2007', coverDate=None, volume=None, issue=None, first=None, last=None, citedbycount=None, type=None, text='Chapman and Hall/CRC, London', fulltext='Alon, U. (2007) An Introduction to Systems Biology: Design Principles of Biolo

In [10]:
article_list = pd.read_csv("C:/Users/jacob/OneDrive - Université Laval/biophilo/Data/pybliometrics_all_bio_arxiv_filtered.csv")
article_list.drop_duplicates()

,Unnamed: 0,eid,doi,pii,pubmed_id,title,subtype,subtypeDescription,creator,afid,...,pageRange,description,authkeywords,citedby_count,openaccess,freetoread,freetoreadLabel,fund_acr,fund_no,fund_sponsor
0,0,2-s2.0-84929582121,10.1021/sb400206c,NaN,24621257.0,Rapidly Characterizing the Fast Dynamics of RN...,ar,Article,Takahashi M.K.,60152345;60143895;60137962;60104946;60089893;6...,...,503-515,RNA regulators are emerging as powerful tools ...,cell-free | response time | RNA genetic circui...,124,1,repositoryam,Green,NSF,1144153,National Science Foundation
1,1,2-s2.0-84903726447,10.1093/bioinformatics/btu121,NaN,24603985.0,PyRAD: Assembly of de novo RADseq loci for phy...,ar,Article,Eaton D.,60030002;60029278,...,1844-1849,Motivation: Restriction-site-associated genomi...,NaN,594,1,repositoryam,Green,NaN,undefined,NaN
2,2,2-s2.0-84908073394,10.1186/2047-217X-3-3,NaN,NaN,A data repository and analysis framework for s...,ar,Article,Eglen S.,60031101;60026712;60016418;60010719;60006222,...,NaN,"Background: During early development, neural c...",Electrophysiology | HDF5 | Multielectrode arra...,24,1,repositoryam,Green,EPSRC,083205,Engineering and Physical Sciences Research Cou...
3,3,2-s2.0-84904251189,10.1371/journal.pone.0090714,NaN,24651479.0,High genetic diversity and adaptive potential ...,ar,Article,Bailey A.L.,60153202;60071676;60032179;60025553;60018179;6...,...,NaN,Key biological properties such as high genetic...,NaN,36,1,repositoryam,Green,OD,P51OD011106,NIH Office of the Director
4,4,2-s2.0-84898619383,10.1371/journal.pone.0091768,NaN,24651046.0,Deciphering the mechanism of β-aminobutyric ac...,ar,Article,Cao H.H.,60031041,...,NaN,The non-protein amino acid β-aminobutyric acid...,NaN,37,1,repositoryam,Green,NaN,undefined,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134761,134761,2-s2.0-85183897401,10.1038/s41467-024-45024-5,NaN,38296951.0,ContScout: sensitive detection and removal of ...,ar,Article,Bálint B.,60212127;60026434;60025038;60014567,...,NaN,Contamination of genomes is an increasingly re...,NaN,8,1,repositoryvor,Green,JGI,758161,Joint Genome Institute
134762,134762,2-s2.0-85198988259,10.1371/journal.pbio.3002711,NaN,39008532.0,Collective peroxide detoxification determines ...,ar,Article,Green R.,60172345;60172343;60016848,...,NaN,Mutagenesis is responsive to many environmenta...,NaN,0,1,repositoryvor,Green,BBSRC,BB/T008725/1,Biotechnology and Biological Sciences Research...
134763,134763,2-s2.0-85192109484,10.1016/j.vaccine.2024.04.073,S0264410X24005140,38704253.0,Evaluation of the precision of the Plasmodium ...,ar,Article,Mertens J.E.,60172675;60123796;60031331;60026851;60017336;6...,...,3621-3629,Recent data indicate increasing disease burden...,Blood-stage malaria antigen | Growth inhibitio...,0,1,repositoryvor,Green,NIAID,undefined,National Institute of Allergy and Infectious D...
134764,134764,2-s2.0-85181741708,10.1002/jez.b.23236,NaN,38192038.0,3D spheroid culturing of Astyanax mexicanus li...,ar,Article,Biswas T.,60028793;60009084,...,301-312,In vitro assays are crucial tools for gaining ...,3D culturing | adaptation | Astyanax mexicanus...,2,1,publisherhybridgold,Hybrid Gold,NSF,1923372,National Science Foundation


In [11]:
from pybliometrics.scopus import AbstractRetrieval
import pandas as pd
import time
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to split EIDs into batches of 1000
def chunk_list(lst, size):
    """Yield successive chunks of a given size from a list."""
    for i in range(0, len(lst), size):
        yield lst[i:i + size]

# Function to fetch references for a single EID
def fetch_references(eid):
    try:
        ref_query = AbstractRetrieval(eid, id_type="eid", view="REF")
        if ref_query.references:
            return [{"id": ref.id, "source_eid": eid, "cited_year": ref.coverDate, "cited_journal" : ref.sourcetitle} for ref in ref_query.references]
    except Exception as e:
        print(f"Error processing EID {eid}: {e}")
    return []

# Process in batches of 1000 EIDs at a time
batch_size = 1000  
eid_list = article_list['eid'].tolist()
citation_data = []

for batch_idx, batch in enumerate(chunk_list(eid_list, batch_size), start=1):
    print(f"\nProcessing batch {batch_idx} ({len(batch)} EIDs)...")

    with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust based on API limits
        future_to_eid = {executor.submit(fetch_references, eid): eid for eid in batch}

        for future in as_completed(future_to_eid):
            citation_data.extend(future.result())  # Append results when ready

    print(f"Batch {batch_idx} complete. Total citations collected: {len(citation_data)}")
    time.sleep(2)  # Optional: Pause to avoid API rate limits

# Convert list to DataFrame once (efficient)
citation_list = pd.DataFrame(citation_data)

print("\nProcessing complete!")
print(citation_list)



Processing batch 1 (1000 EIDs)...
Batch 1 complete. Total citations collected: 54311

Processing batch 2 (1000 EIDs)...
Batch 2 complete. Total citations collected: 107333

Processing batch 3 (1000 EIDs)...
Batch 3 complete. Total citations collected: 165866

Processing batch 4 (1000 EIDs)...
Batch 4 complete. Total citations collected: 220975

Processing batch 5 (1000 EIDs)...
Batch 5 complete. Total citations collected: 278149

Processing batch 6 (1000 EIDs)...
Batch 6 complete. Total citations collected: 337924

Processing batch 7 (1000 EIDs)...
Batch 7 complete. Total citations collected: 395967

Processing batch 8 (1000 EIDs)...
Batch 8 complete. Total citations collected: 451915

Processing batch 9 (1000 EIDs)...
Batch 9 complete. Total citations collected: 510299

Processing batch 10 (1000 EIDs)...
Batch 10 complete. Total citations collected: 566724

Processing batch 11 (1000 EIDs)...
Batch 11 complete. Total citations collected: 621852

Processing batch 12 (1000 EIDs)...
Batc

In [13]:
citation_data

[{'id': '0000395622',
  'source_eid': '2-s2.0-84896855185',
  'cited_year': '1996-01-01',
  'cited_journal': 'Physical Review Letters'},
 {'id': '15744387380',
  'source_eid': '2-s2.0-84896855185',
  'cited_year': '2005-01-01',
  'cited_journal': 'Current Opinion in Genetics and Development'},
 {'id': '33846516584',
  'source_eid': '2-s2.0-84896855185',
  'cited_year': None,
  'cited_journal': 'Pattern recognition and machine learning'},
 {'id': '84889281816',
  'source_eid': '2-s2.0-84896855185',
  'cited_year': '2005-04-07',
  'cited_journal': 'Elements of Information Theory'},
 {'id': '60949114079',
  'source_eid': '2-s2.0-84896855185',
  'cited_year': None,
  'cited_journal': 'Information theory and statistics: A tutorial'},
 {'id': '35348955882',
  'source_eid': '2-s2.0-84896855185',
  'cited_year': '2007-10-26',
  'cited_journal': 'Molecular Cell'},
 {'id': '0037119587',
  'source_eid': '2-s2.0-84896855185',
  'cited_year': '2002-08-16',
  'cited_journal': 'Science'},
 {'id': '34

In [12]:
citation_list.to_csv("C:/Users/jacob/OneDrive - Université Laval/biophilo/Data/refs_all_bio_arxiv.csv")